# Organize Hunting & Fishing License Data
a. stein 2.3.22

In [1]:
%pylab inline
import pandas as pd
import xarray as xr
import os
from tqdm.notebook import tqdm

Populating the interactive namespace from numpy and matplotlib


In [2]:
# point to the appropriate directory
dir = '/pool0/home/steinadi/data/drought/DatabaseDrafts/HuntingLicenses_OR'
# and just double check that this directory looks right
!ls $dir

'2016 AnglingDailyLicenses.csv'        '2018 ResidentAnglerLicenses.csv'
'2016 AnglingLicenseEndorsements.csv'  '2018 ResidentComboLicenses.csv'
'2016 BirdHunterLicenses.csv'	       '2018 ResidentHunterLicenses.csv'
'2016 DisabledVetComboLicenses.csv'    '2018 SeniorAnglerLicenses.csv'
'2016 JuvenileAnglerLicenses.csv'      '2018 SeniorComboLicenses.csv'
'2016 JuvenileHunterLicenses.csv'      '2018 SeniorHunterLicenses.csv'
'2016 NonResidentAnglerLicenses.csv'   '2018 ShellfishLicenses.csv'
'2016 NonResidentHunterLicenses.csv'   '2018 SportsPackLicenses.csv'
'2016 ResidentAnglerLicenses.csv'      '2019 AnglingDailyLicenses.csv'
'2016 ResidentComboLicenses.csv'       '2019 AnglingLicenseEndorsements.csv'
'2016 ResidentHunterLicenses.csv'      '2019 BirdHunterLicenses.csv'
'2016 SeniorAnglerLicenses.csv'        '2019 DisabledVetComboLicenses.csv'
'2016 SeniorComboLicenses.csv'	       '2019 EducationClasses.csv'
'2016 SeniorHunterLicenses.csv'        '2019 JuvenileAnglerLicenses.csv'
'201

Okay, now to do a series of checks to figure out data compatibility so we can eventually put it all in one file

In [3]:
# want to be able to parse through these file names later
file_names = os.listdir(dir)
# let's get our list of unique names, disregarding the year 
file_names_dateless = [name[5:] for name in file_names]
file_years = ['2016','2017','2018','2019','2020']
file_names_unique = list(set(file_names_dateless))
print(file_years)
print(file_names_unique)

['2016', '2017', '2018', '2019', '2020']
['JuvenileHunterLicenses.csv', 'AnglingDailyLicenses.csv', 'ShellfishLicenses.csv', 'NonResidentHunterLicenses.csv', 'SeniorComboLicenses.csv', 'EducationClasses.csv', 'ResidentHunterLicenses.csv', 'ResidentComboLicenses.csv', 'SeniorHunterLicenses.csv', 'JuvenileAnglerLicenses.csv', 'DisabledVetComboLicenses.csv', 'SportsPackLicenses.csv', 'ResidentAnglerLicenses.csv', 'SeniorAnglerLicenses.csv', 'TwoRodAnglerLicenses.csv', 'AnglingLicenseEndorsements.csv', 'NonResidentAnglerLicenses.csv', 'BirdHunterLicenses.csv']


In [4]:
# let's check if the types of data stored in each csv is different at all
# (includes different names, different lengths, different order)

# but first need to grab something to compare, see after this cell
# for why I pick this file
test_0 = pd.read_csv(f'{dir}/{file_years[0]} {file_names_unique[1]}')

for file_name in file_names_unique:
    print(file_name)
    try:
        test_i = pd.read_csv(f'{dir}/{file_years[0]} {file_name}')
        col_dif = len(np.where(test_0.columns != test_i.columns)[0])
        print(f'Col Dif = {col_dif}')
        del test_i
    except:
        print("File Not Found")

JuvenileHunterLicenses.csv
Col Dif = 0
AnglingDailyLicenses.csv
Col Dif = 0
ShellfishLicenses.csv
Col Dif = 0
NonResidentHunterLicenses.csv
Col Dif = 0
SeniorComboLicenses.csv
Col Dif = 0
EducationClasses.csv
File Not Found
ResidentHunterLicenses.csv
Col Dif = 0
ResidentComboLicenses.csv


/pool0/data/steinadi/.conda/envs/sedi/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Col Dif = 1
SeniorHunterLicenses.csv
Col Dif = 0
JuvenileAnglerLicenses.csv
Col Dif = 0
DisabledVetComboLicenses.csv
Col Dif = 0
SportsPackLicenses.csv
Col Dif = 0
ResidentAnglerLicenses.csv
Col Dif = 0
SeniorAnglerLicenses.csv
Col Dif = 0
TwoRodAnglerLicenses.csv
Col Dif = 0
AnglingLicenseEndorsements.csv
Col Dif = 0
NonResidentAnglerLicenses.csv
Col Dif = 0
BirdHunterLicenses.csv
Col Dif = 0


In [5]:
#pd.read_csv(f'{dir}/{file_years[0]} {file_names_unique[0]}').columns

Okay, I think 'Unamed: 6' should be 'License Type'

In [6]:
pd.read_csv(f'{dir}/{file_years[0]} {file_names_unique[1]}').columns

Index(['NAME', 'ADDRESS', 'CITY', 'STATE', 'ZIP', 'PHONE NUMBER',
       'LICENSE TYPE', 'LICENSE DESCRIPTION', 'ISSUE DATE', 'RESIDENCY',
       'GENDER', 'AGE'],
      dtype='object')

In [7]:
#pd.read_csv(f'{dir}/{file_years[0]} {file_names_unique[0]}')['Unnamed: 6']

In [8]:
pd.read_csv(f'{dir}/{file_years[0]} {file_names_unique[1]}')['LICENSE TYPE']

0         21009
1         21003
2         21004
3         21004
4         21004
          ...  
144687    21003
144688    21003
144689    21004
144690    21003
144691    21003
Name: LICENSE TYPE, Length: 144692, dtype: int64

Yeah those look pretty similar. So we appear to have rather nice data actually, having the same info collected for each.

In [9]:
data_cols = list(pd.read_csv(f'{dir}/{file_years[0]} {file_names_unique[1]}').columns)
data_cols

['NAME',
 'ADDRESS',
 'CITY',
 'STATE',
 'ZIP',
 'PHONE NUMBER',
 'LICENSE TYPE',
 'LICENSE DESCRIPTION',
 'ISSUE DATE',
 'RESIDENCY',
 'GENDER',
 'AGE']

Let's think about what data we actually need to preserve.

**'NAME'** - could be useful to distinguish records   
**'ADDRESS'** - hmm .. could be useful for some geospatial analysis maybe    
**'CITY'** - see ADDRESS
**'STATE'** - I mean I know it's OR, but we may want to add WA later    
**'ZIP'** - yeah sure, see ADDRESS    
**'PHONE NUMBER'** - don't need this identifier, we ain't going to be calling folks    
**'LICENSE TYPE'** - could be useful later for seperating    
**'LICENSE DESCRIPTION'** - yeah sure    
**'ISSUE DATE'** - most surely this can be helpful for timeseries analysis at least   
**'RESIDENCEY'** - don't think so ...
**'GENDER'** - I don't really want to do a gender analysis
**'AGE'** - also not really the data we are looking for

So, let's save the following: **'NAME'**, **'ADDRESS'**, **'CITY'**, **'STATE'**, **'ZIP'**, **'LICENSE TYPE'**, **'LICENSE DESCRIPTION'**, **'ISSUE DATE'**

In [10]:
# going to do these by hand just to make life simple

data_cols_refine = ['NAME', 'ADDRESS', 'CITY', 'STATE', 'ZIP', 'LICENSE TYPE', 'LICENSE DESCRIPTION', 'ISSUE DATE']
data_cols_refine_dex = [0, 1, 2, 3, 4, 6, 7, 8]

Now I need an xarray Dataset that incorporates the different file names, the data columns, and year ...

In [11]:
# need to find what can be used as unique identifiers
for col in data_cols:
    test = len(set(test_0[col].values)) == len(test_0[col].values)
    print(f'{col} unique ... {test}')

NAME unique ... False
ADDRESS unique ... False
CITY unique ... False
STATE unique ... False
ZIP unique ... False
PHONE NUMBER unique ... False
LICENSE TYPE unique ... False
LICENSE DESCRIPTION unique ... False
ISSUE DATE unique ... False
RESIDENCY unique ... False
GENDER unique ... False
AGE unique ... False


Great. So we don't have a purely unique identifier since I'm betting multiple licenses are gathered either for the same residence or the same day or ... whatever. So we will likely need to either assign some id code to each indiviudal one and not use one columen as the coordinate for the xarray. Maybe this is better overall suited for pandas then and leave xarray for the statistics on them.

Let me check whether I need to keep the files seperate since license type is built into them

In [12]:
print(file_names_unique[1])
test_0.head()

AnglingDailyLicenses.csv


,NAME,ADDRESS,CITY,STATE,ZIP,PHONE NUMBER,LICENSE TYPE,LICENSE DESCRIPTION,ISSUE DATE,RESIDENCY,GENDER,AGE
0,"DORROUGH, KRISTEN",18305 SE NEWPORT WAY,ISSAQUAH,WA,98027,0.000000e+00,21009,SEVEN DAY ANGLING,7/10/2016,Unknown,F,44
1,NaN,NaN,NaN,NaN,NaN,NaN,21003,ONE DAY ANGLING LICENSE,7/7/2016,Unknown,M,14
2,"CAMBELL, JAMES",614 N BELLIN RD,IDAHO FALLS,ID,83402,2.087163e+09,21004,TWO DAY ANGLING LICENSE,6/6/2016,Unknown,M,48
3,"DEPORTO, RONALD",677 RUBY DR,VACAVILLE,CA,95687,0.000000e+00,21004,TWO DAY ANGLING LICENSE,9/17/2016,Unknown,M,56
4,"HERNANDEZ, RESENDIZ",3952 PARTIDGE LN NE,SALEM,OR,97303,0.000000e+00,21004,TWO DAY ANGLING LICENSE,5/27/2016,Unknown,M,29


It looks like there are some slight variations, but **'LICENSE DESCRIPTION'** takes care of the detail I need.

Okay, let's add a year column for each and put them all in one ...

Or actually, don't need to save the year cause we have issue dates!

In [33]:
license_df = pd.DataFrame(columns=data_cols_refine)
t = tqdm(total=(len(file_years)*len(file_names_unique)))
for year in file_years:
    for file_name in file_names_unique:
        # don't select out columns yet ... some have data in the wrong
        # spots despite column names matching. ugh.
        file = f'{year} {file_name}'
        t.set_description(file)
        file_path = f'{dir}/{file}'
        if os.path.exists(file_path):
            load_csv = pd.read_csv(file_path)
            load_csv_cols = load_csv.columns
            # went through all the columns this pops up, appears to
            # be some strange artifact, we don't loose anything but
            # cutting it out
            if 'Unnamed: 12' in load_csv_cols:
                print(f'{file} has Unnamed: 12')
                del load_csv['Unnamed: 12']
                load_csv_cols = load_csv.columns
            if len(np.where(data_cols != load_csv_cols)[0]) != 0:
                # this means a column name got misread, but is 
                # still going to be in same order
                print(f'{file} ... fixing naming')
                rename_dict = dict()
                for col, d_col in zip(load_csv_cols, data_cols):
                    rename_dict[col] = d_col
                load_csv.rename(columns=rename_dict, inplace=True)
                del rename_dict
            license_df = license_df.append(load_csv)
            del load_csv
        t.update()
        #except:
        #    print(f'{file_path} ... does not exist')
        #    pass
license_df

  0%|          | 0/90 [00:00<?, ?it/s]

/pool0/data/steinadi/.conda/envs/sedi/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


2016 ResidentComboLicenses.csv ... fixing naming
2018 NonResidentHunterLicenses.csv has Unnamed: 12


/pool0/data/steinadi/.conda/envs/sedi/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


2018 SeniorComboLicenses.csv has Unnamed: 12
2018 SeniorHunterLicenses.csv has Unnamed: 12
2018 DisabledVetComboLicenses.csv has Unnamed: 12
2018 SeniorAnglerLicenses.csv has Unnamed: 12


/pool0/data/steinadi/.conda/envs/sedi/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


2018 NonResidentAnglerLicenses.csv has Unnamed: 12
2019 NonResidentHunterLicenses.csv has Unnamed: 12
2019 SeniorComboLicenses.csv has Unnamed: 12
2019 EducationClasses.csv has Unnamed: 12
2019 ResidentHunterLicenses.csv has Unnamed: 12
2019 SeniorHunterLicenses.csv has Unnamed: 12
2019 DisabledVetComboLicenses.csv has Unnamed: 12
2019 SeniorAnglerLicenses.csv has Unnamed: 12
2019 NonResidentAnglerLicenses.csv has Unnamed: 12
2020 SeniorComboLicenses.csv has Unnamed: 12
2020 SeniorHunterLicenses.csv has Unnamed: 12
2020 DisabledVetComboLicenses.csv has Unnamed: 12
2020 SeniorAnglerLicenses.csv has Unnamed: 12


,NAME,ADDRESS,CITY,STATE,ZIP,LICENSE TYPE,LICENSE DESCRIPTION,ISSUE DATE,PHONE NUMBER,RESIDENCY,GENDER,AGE
0,NaN,NaN,NaN,NaN,NaN,10138,YOUTH UNDER 12 SMALL GAME HUNTING,1/16/2016,NaN,Unknown,M,11.0
1,NaN,NaN,NaN,NaN,NaN,10138,YOUTH UNDER 12 SMALL GAME HUNTING,4/24/2016,NaN,Unknown,M,10.0
2,NaN,NaN,NaN,NaN,NaN,10138,YOUTH UNDER 12 SMALL GAME HUNTING,6/21/2016,NaN,Unknown,M,10.0
3,NaN,NaN,NaN,NaN,NaN,10138,YOUTH UNDER 12 SMALL GAME HUNTING,5/15/2016,NaN,Unknown,M,12.0
4,NaN,NaN,NaN,NaN,NaN,10138,YOUTH UNDER 12 SMALL GAME HUNTING,4/24/2016,NaN,Unknown,F,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3075,"zilch, mark",3176 marguerite ave,CORNING,California,96021,155,Three Day Bird Hunting License,10/9/2020,0.0,Non-Resident,Male,34.0
3076,"Zimmer, Joshua A",1555 Virginia Way,NAPA,California,94558,155,Three Day Bird Hunting License,2/18/2020,0.0,Non-Resident,Male,34.0
3077,"Zimmerman, Michael E",PO Box 234,RIVERSIDE,Washington,98849,155,Three Day Bird Hunting License,8/11/2020,0.0,Non-Resident,Male,62.0
3078,"zinkin, dewayne",5 e river park place w ste 203,FRESNO,California,93720,155,Three Day Bird Hunting License,5/12/2020,0.0,Non-Resident,Male,78.0


In [34]:
license_df.index

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            3070, 3071, 3072, 3073, 3074, 3075, 3076, 3077, 3078, 3079],
           dtype='int64', length=5310659)

Okay, there are wayyy more values (5310659, including this number to make sure we don't loose data in future) than the index counts up too ... I'm betting that the index was made with each load ...

In [35]:
# checking theory
license_df.loc[8].head()

,NAME,ADDRESS,CITY,STATE,ZIP,LICENSE TYPE,LICENSE DESCRIPTION,ISSUE DATE,PHONE NUMBER,RESIDENCY,GENDER,AGE
8,NaN,NaN,NaN,NaN,NaN,10138,YOUTH UNDER 12 SMALL GAME HUNTING,5/15/2016,NaN,Unknown,M,10.0
8,"AABY, ROYAL C",2735 ELISSAGARAY DR,TRACY,CA,95304,21004,TWO DAY ANGLING LICENSE,10/6/2016,2098355869.0,Unknown,M,67.0
8,"AALTO, BENITO",49034 HWY 101,BANDON,OR,97411,10565,DISABLED VET SHELLFISH,2/6/2016,4258024073.0,Unknown,M,68.0
8,"CHASE, TY R",4520 HARRY CASH RD,MONTAGUE,CA,96064,10203,NON-RESIDENT HUNTING,5/14/2016,5303402317.0,Unknown,M,19.0
8,"ACKERMAN, PATRICK H",3998 78TH AVE SE,SALEM,OR,97317,10164,SENIOR COMBINATION WITH COLUMBIA BASIN ENDORS...,12/16/2015,5035814600.0,Unknown,M,77.0


Yup. Well at least it looks like it wasn't overwritten. Let's reset that index

In [36]:
license_df = license_df.reset_index()
del license_df['index']
license_df

,NAME,ADDRESS,CITY,STATE,ZIP,LICENSE TYPE,LICENSE DESCRIPTION,ISSUE DATE,PHONE NUMBER,RESIDENCY,GENDER,AGE
0,NaN,NaN,NaN,NaN,NaN,10138,YOUTH UNDER 12 SMALL GAME HUNTING,1/16/2016,NaN,Unknown,M,11.0
1,NaN,NaN,NaN,NaN,NaN,10138,YOUTH UNDER 12 SMALL GAME HUNTING,4/24/2016,NaN,Unknown,M,10.0
2,NaN,NaN,NaN,NaN,NaN,10138,YOUTH UNDER 12 SMALL GAME HUNTING,6/21/2016,NaN,Unknown,M,10.0
3,NaN,NaN,NaN,NaN,NaN,10138,YOUTH UNDER 12 SMALL GAME HUNTING,5/15/2016,NaN,Unknown,M,12.0
4,NaN,NaN,NaN,NaN,NaN,10138,YOUTH UNDER 12 SMALL GAME HUNTING,4/24/2016,NaN,Unknown,F,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5310654,"zilch, mark",3176 marguerite ave,CORNING,California,96021,155,Three Day Bird Hunting License,10/9/2020,0.0,Non-Resident,Male,34.0
5310655,"Zimmer, Joshua A",1555 Virginia Way,NAPA,California,94558,155,Three Day Bird Hunting License,2/18/2020,0.0,Non-Resident,Male,34.0
5310656,"Zimmerman, Michael E",PO Box 234,RIVERSIDE,Washington,98849,155,Three Day Bird Hunting License,8/11/2020,0.0,Non-Resident,Male,62.0
5310657,"zinkin, dewayne",5 e river park place w ste 203,FRESNO,California,93720,155,Three Day Bird Hunting License,5/12/2020,0.0,Non-Resident,Male,78.0


I have also found out that not all the data is in the right spots (I think this is why the mixed type errors get thrown), arg. Let's fix that before doing more cleaning.

Let's see ... well we can try checking whether each column is purely that datatype ... here is what I THINK each should be ...

NAME - str    
ADDRESS - str    
CITY - str    
STATE - str    
ZIP - str (because it was likely parsed from a str address)    
LICENSE TYPE - int    
LICENSE DESCRIPTION - str    
PHONE NUMBER - int or float    
RESIDENCEY - str    
GENDER - str 
AGE - float

In [57]:
# check how many of each type are in each column
for col in data_cols:
    nan_count = 0
    str_count = 0
    int_count = 0
    float_count = 0
    other_count = 0
    for val in license_df[col].values:
        if isinstance(val, (int, float)):
            if np.isnan(val):
                nan_count += 1
            elif isinstance(val, int):
                int_count += 1
            elif isinstance(val, float):
                float_count += 1
        elif isinstance(val, str):
            str_count += 1
        else:
            other_count += 1
    
    print(f'{col}: {nan_count} NaN, {int_count} int, {float_count} float, {str_count} str, {other_count} other')
        

NAME: 606387 NaN, 0 int, 0 float, 4704272 str, 0 other
ADDRESS: 676812 NaN, 0 int, 0 float, 4633847 str, 0 other
CITY: 676813 NaN, 0 int, 0 float, 4633846 str, 0 other
STATE: 676813 NaN, 0 int, 0 float, 4633846 str, 0 other
ZIP: 676818 NaN, 150127 int, 769257 float, 3714457 str, 0 other
PHONE NUMBER: 606386 NaN, 365097 int, 4170931 float, 168245 str, 0 other
LICENSE TYPE: 7 NaN, 3426921 int, 0 float, 1883731 str, 0 other
LICENSE DESCRIPTION: 7 NaN, 0 int, 0 float, 5310652 str, 0 other
ISSUE DATE: 7 NaN, 0 int, 0 float, 5310652 str, 0 other
RESIDENCY: 7 NaN, 0 int, 0 float, 5310652 str, 0 other
GENDER: 70446 NaN, 0 int, 0 float, 5240213 str, 0 other
AGE: 70463 NaN, 0 int, 5240196 float, 0 str, 0 other


Great that we captured all the datatypes there, nothing else strange.

**NAME**, **ADDRESS**, **CITY**, and **STATE** look good to the point I don't want to bother checking ...

I care most about **LICENESE TYPE**, **LICENSE DESCRIPTION**, and **ISSUE DATE** being right. Their mismatches might be in other columns but let's see if I can figure out if it's only a few columns messed up ...

In [112]:
catch_lt = []
catch_lt_dex = []
for i, val in enumerate(license_df['LICENSE TYPE']):
    # these should just be numbers, so if there is whitespace
    # i am betting it's in the wrong column
    if isinstance(val, str) and ' ' in val:
        catch_lt.append(val)
        catch_lt_dex.append(i)
catch_lt

['Annual Combination License',
 'Annual Hunting License',
 'Annual Angling License',
 'Columbia River Basin Endorsement (Free)',
 'One Day Angling License',
 'Annual Shellfish License',
 'Annual Combination License',
 'Annual Combination License',
 'Annual Angling License',
 'Annual Angling License',
 'Columbia River Basin Endorsement (Free)',
 'Annual Angling License',
 'Annual Combination License',
 'Annual Hunting License',
 'Two Rod Endorsement',
 'Columbia River Basin Endorsement (Free)']

Ah-ha! Got some. Let's look closer ...

In [129]:
license_df.loc[catch_lt_dex]

,NAME,ADDRESS,CITY,STATE,ZIP,LICENSE TYPE,LICENSE DESCRIPTION,ISSUE DATE,PHONE NUMBER,RESIDENCY,GENDER,AGE,ISSUE DATETIME
2243476,"UDING, JOHN "",P.O. BOX 195""",CRESCENT,Oregon,97733,0,Annual Combination License,01/15/2018,Resident,101.0,Male,65,NaN,NaT
2315592,"Slate, Robert","22401 S.E. Morrison st"",Gresham""",Oregon,97030,5037583755,Annual Hunting License,5/8/2018,Resident,150.0,Male,58,NaN,NaT
2765251,"UDING, JOHN "",P.O. BOX 195""",CRESCENT,Oregon,97733,0,Annual Angling License,01/15/2018,Resident,120.0,Male,65,NaN,NaT
2857314,"UDING, JOHN "",P.O. BOX 195""",CRESCENT,Oregon,97733,0,Columbia River Basin Endorsement (Free),1/15/2018,Resident,0136B,Male,65,NaN,NaT
2963288,"Lee, Herschel "",4295 SE ROSE PETAL PL""",Hillsboro,Oregon,97123,0,One Day Angling License,7/27/2019,Resident,130.0,Male,63,NaN,NaT
3151285,"Lee, Herschel "",4295 SE ROSE PETAL PL""",Hillsboro,Oregon,97123,0,Annual Shellfish License,7/9/2019,Resident,0140A,Male,63,NaN,NaT
3342162,"UDING, JOHN "",P.O. BOX 195""",CRESCENT,Oregon,97733,0,Annual Combination License,01/16/2019,Resident,101,Male,66,NaN,NaT
3492995,"Penalba, Marialice "",10855 SW 60th ave""",PORTLAND,Oregon,97219,0,Annual Combination License,2/20/2019,Resident,101.0,Female,54,NaN,NaT
3622436,"Bricker, James "",348 Ridgewood Drive""",JACKSONVILLE,Oregon,97530,0,Annual Angling License,3/24/2019,Resident,120.0,Male,36,NaN,NaT
3702531,"Lee, Herschel "",4295 SE ROSE PETAL PL""",Hillsboro,Oregon,97123,0,Annual Angling License,7/9/2019,Resident,120.0,Male,63,NaN,NaT


Oh boy, that's all sorts of a mess. Here is what I'm seeing needs to happen:

All error:    
Gender --> Age    
Residencey --> Gender    
Issue Date --> Residencey    
License Description --> Issue Date    
Licenese Type --> License Description    
Phone Number --> License Type    

Address parsing error:    
Zip --> Phone Number    
State --> Zip    
City --> State    
Address --> City    
Name --> Name & Address for some

In [176]:
repair_license_df = license_df.loc[catch_lt_dex].copy()
repair_license_df['AGE'] = repair_license_df['GENDER']
repair_license_df['GENDER'] = repair_license_df['RESIDENCY']
repair_license_df['RESIDENCY'] = repair_license_df['ISSUE DATE']
repair_license_df['ISSUE DATE'] = repair_license_df['LICENSE DESCRIPTION']
repair_license_df['LICENSE DESCRIPTION'] = repair_license_df['LICENSE TYPE']
repair_license_df['LICENSE TYPE'] = repair_license_df['PHONE NUMBER']

repair_license_df['PHONE NUMBER'] = repair_license_df['ZIP']
repair_license_df['ZIP'] = repair_license_df['STATE']
repair_license_df['STATE'] = repair_license_df['CITY']
repair_license_df['CITY'] = repair_license_df['ADDRESS']
repair_license_df.loc[[2315592, 4481604],'CITY'] = ['Gresham', 'Gresham']
repair_license_df[['NAME','ADDRESS']] = repair_license_df['NAME'].str.split('",', expand=True)
repair_license_df.loc[[2315592, 4481604],'ADDRESS'] ='22401 S.E. Morrison st"'
# this last bit just cuts of the tailing "
repair_license_df['ADDRESS'] = [val[:-1] for val in repair_license_df['ADDRESS'].values]

# check work
repair_license_df

,NAME,ADDRESS,CITY,STATE,ZIP,LICENSE TYPE,LICENSE DESCRIPTION,ISSUE DATE,PHONE NUMBER,RESIDENCY,GENDER,AGE,ISSUE DATETIME
2243476,"UDING, JOHN",P.O. BOX 195,CRESCENT,Oregon,97733,101.0,Annual Combination License,01/15/2018,0,Resident,Male,65,NaT
2315592,"Slate, Robert",22401 S.E. Morrison st,Gresham,Oregon,97030,150.0,Annual Hunting License,5/8/2018,5037583755,Resident,Male,58,NaT
2765251,"UDING, JOHN",P.O. BOX 195,CRESCENT,Oregon,97733,120.0,Annual Angling License,01/15/2018,0,Resident,Male,65,NaT
2857314,"UDING, JOHN",P.O. BOX 195,CRESCENT,Oregon,97733,0136B,Columbia River Basin Endorsement (Free),1/15/2018,0,Resident,Male,65,NaT
2963288,"Lee, Herschel",4295 SE ROSE PETAL PL,Hillsboro,Oregon,97123,130.0,One Day Angling License,7/27/2019,0,Resident,Male,63,NaT
3151285,"Lee, Herschel",4295 SE ROSE PETAL PL,Hillsboro,Oregon,97123,0140A,Annual Shellfish License,7/9/2019,0,Resident,Male,63,NaT
3342162,"UDING, JOHN",P.O. BOX 195,CRESCENT,Oregon,97733,101,Annual Combination License,01/16/2019,0,Resident,Male,66,NaT
3492995,"Penalba, Marialice",10855 SW 60th ave,PORTLAND,Oregon,97219,101.0,Annual Combination License,2/20/2019,0,Resident,Female,54,NaT
3622436,"Bricker, James",348 Ridgewood Drive,JACKSONVILLE,Oregon,97530,120.0,Annual Angling License,3/24/2019,0,Resident,Male,36,NaT
3702531,"Lee, Herschel",4295 SE ROSE PETAL PL,Hillsboro,Oregon,97123,120.0,Annual Angling License,7/9/2019,0,Resident,Male,63,NaT


In [177]:
# nice, let's add that back in
license_df.loc[catch_lt_dex] = repair_license_df

In [178]:
catch_id = []
catch_id_dex = []
for i, val in enumerate(license_df['ISSUE DATE']):
    # these should all be m/d/y dates, so if they
    # do not have those slashes then I'm betting
    # that it's not right
    if isinstance(val, str) and not '/' in val:
        catch_id.append(val)
        catch_id_dex.append(i)
catch_id

['One Day Angling License', 'One Day Angling License']

In [184]:
license_df.loc[catch_id_dex]

,NAME,ADDRESS,CITY,STATE,ZIP,LICENSE TYPE,LICENSE DESCRIPTION,ISSUE DATE,PHONE NUMBER,RESIDENCY,GENDER,AGE,ISSUE DATETIME
2984693,ODORISIO,"LIZ""",NaN,NaN,NaN,0,130,One Day Angling License,NaN,9/15/2019,Unknown,NaN,NaT
2984694,ODORISIO,"NATE""",NaN,NaN,NaN,0,130,One Day Angling License,NaN,9/15/2019,Unknown,NaN,NaT


Alright, not as many issues, but still need to reorder. 

License Type --> Phone Number    
License Description --> Licenese Type    
Issue Date --> License Description    
Residencey --> Issue Date

Address + Name --> Name    
Nan --> Address

In [188]:
repair_license_df_2 = license_df.loc[catch_id_dex].copy()
repair_license_df_2['PHONE NUMBER'] = repair_license_df_2['LICENSE TYPE']
repair_license_df_2['LICENSE TYPE'] = repair_license_df_2['LICENSE DESCRIPTION']
repair_license_df_2['LICENSE DESCRIPTION'] = repair_license_df_2['ISSUE DATE']
repair_license_df_2['ISSUE DATE'] = repair_license_df_2['RESIDENCY']
repair_license_df_2['RESIDENCY'] = [pd.NaT, pd.NaT]

repair_license_df_2['NAME'] = ['ODORISO, LIZ', 'ODORISO, NATE']
repair_license_df_2['ADDRESS'] = [pd.NaT, pd.NaT]

# check work
repair_license_df_2

,NAME,ADDRESS,CITY,STATE,ZIP,LICENSE TYPE,LICENSE DESCRIPTION,ISSUE DATE,PHONE NUMBER,RESIDENCY,GENDER,AGE,ISSUE DATETIME
2984693,"ODORISO, LIZ",NaT,NaN,NaN,NaN,130,One Day Angling License,9/15/2019,0,NaT,Unknown,NaN,NaT
2984694,"ODORISO, NATE",NaT,NaN,NaN,NaN,130,One Day Angling License,9/15/2019,0,NaT,Unknown,NaN,NaT


In [189]:
license_df.loc[catch_id_dex] = repair_license_df_2

In [193]:
catch_id_other = 2960230	
license_df.loc[catch_id_other]

NAME                   Kruse, Alex ",",",",",0"
ADDRESS                                     130
CITY                    One Day Angling License
STATE                                 7/14/2019
ZIP                                     Unknown
LICENSE TYPE                                NaN
LICENSE DESCRIPTION                         NaN
ISSUE DATE                                  NaN
PHONE NUMBER                                NaN
RESIDENCY                                   NaN
GENDER                                      NaN
AGE                                         NaN
ISSUE DATETIME                              NaT
Name: 2960230, dtype: object

In [196]:
repair_license_df_3 = license_df.loc[catch_id_other].copy()
repair_license_df_3['GENDER'] = repair_license_df_3['ZIP']
repair_license_df_3['ISSUE DATE'] = repair_license_df_3['STATE']
repair_license_df_3['LICENSE DESCRIPTION'] = repair_license_df_3['CITY']
repair_license_df_3['LICENSE TYPE'] = repair_license_df_3['ADDRESS']
repair_license_df_3['NAME'] = 'Kruse, Alex'

for val in ['ADDRESS', 'CITY', 'STATE', 'ZIP']:
    repair_license_df_3[val] = np.nan

# check work
repair_license_df_3

NAME                               Kruse, Alex
ADDRESS                                    NaN
CITY                                       NaN
STATE                                      NaN
ZIP                                        NaN
LICENSE TYPE                               130
LICENSE DESCRIPTION    One Day Angling License
ISSUE DATE                           7/14/2019
PHONE NUMBER                               NaN
RESIDENCY                                  NaN
GENDER                                 Unknown
AGE                                        NaN
ISSUE DATETIME                             NaT
Name: 2960230, dtype: object

In [197]:
license_df.loc[catch_id_other] = repair_license_df_3

That's better, now we can get into cleaning up the **'ISSUE DATE'**

In [198]:
# need to set the errors to coerce to flag them as things we shouldn't be plottin then ...
# just using ignore will stop it from parsing what can be parsed
license_df['ISSUE DATETIME'] = pd.to_datetime(license_df['ISSUE DATE'], format='%m/%d/%Y', errors='coerce')
license_df.head()

,NAME,ADDRESS,CITY,STATE,ZIP,LICENSE TYPE,LICENSE DESCRIPTION,ISSUE DATE,PHONE NUMBER,RESIDENCY,GENDER,AGE,ISSUE DATETIME
0,NaN,NaN,NaN,NaN,NaN,10138,YOUTH UNDER 12 SMALL GAME HUNTING,1/16/2016,NaN,Unknown,M,11.0,2016-01-16
1,NaN,NaN,NaN,NaN,NaN,10138,YOUTH UNDER 12 SMALL GAME HUNTING,4/24/2016,NaN,Unknown,M,10.0,2016-04-24
2,NaN,NaN,NaN,NaN,NaN,10138,YOUTH UNDER 12 SMALL GAME HUNTING,6/21/2016,NaN,Unknown,M,10.0,2016-06-21
3,NaN,NaN,NaN,NaN,NaN,10138,YOUTH UNDER 12 SMALL GAME HUNTING,5/15/2016,NaN,Unknown,M,12.0,2016-05-15
4,NaN,NaN,NaN,NaN,NaN,10138,YOUTH UNDER 12 SMALL GAME HUNTING,4/24/2016,NaN,Unknown,F,9.0,2016-04-24


In [199]:
license_df_time_sorted = license_df.sort_values('ISSUE DATETIME')
# and let's actually name those id's something arbitrary
license_df_time_sorted.index.set_names('id', inplace=True)
license_df_time_sorted

,NAME,ADDRESS,CITY,STATE,ZIP,LICENSE TYPE,LICENSE DESCRIPTION,ISSUE DATE,PHONE NUMBER,RESIDENCY,GENDER,AGE,ISSUE DATETIME
id,,,,,,,,,,,,,
275220,"ROGERS, VERLA M",1646 KING WAY,WOODBURN,OR,97071,10160,ANNUAL SHELLFISH,12/1/2015,5039824022.0,Unknown,F,82.0,2015-12-01
249706,"MITCHELL, CHARLES L",1842 CHERRY ST,UMATILLA,OR,97882,10160,ANNUAL SHELLFISH,12/1/2015,5419226294.0,Unknown,M,71.0,2015-12-01
563279,"SCOTT, MICHAEL W",64643 BOONES BOROUGH DR,BEND,OR,97701.0,10574,PIONEER COMBINATION WITH COLUMBIA BASIN ENDORS...,12/1/2015,5413890112.0,Unknown,M,69.0,2015-12-01
250978,"MOORE, LAWRENCE H",5247 TERESA WAY,CENTRAL POINT,OR,97502,10565,DISABLED VET SHELLFISH,12/1/2015,5416644441.0,Unknown,M,60.0,2015-12-01
879870,"SIMMONS, WILLIAM R",623 DRIFTWOOD,EUGENE,OR,97402.0,10573,WHEELCHAIR ANGLING WITH COLUMBIA BASIN ENDORS...,12/1/2015,5414630518.0,Unknown,M,53.0,2015-12-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2453007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3531732,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3532463,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [200]:
license_df_time_sorted[pd.isnull(license_df_time_sorted['ISSUE DATETIME'])]

,NAME,ADDRESS,CITY,STATE,ZIP,LICENSE TYPE,LICENSE DESCRIPTION,ISSUE DATE,PHONE NUMBER,RESIDENCY,GENDER,AGE,ISSUE DATETIME
id,,,,,,,,,,,,,
2452855,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2453007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3531732,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3532463,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3536331,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3538199,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


Finally, an acceptable set of empy values. We should just remove these.

In [203]:
remove_id = [2452855, 2453007, 3531732, 3532463, 3536331, 3538199]
license_df_time_sorted = license_df_time_sorted.drop(remove_id)

In [204]:
license_df_time_sorted[pd.isnull(license_df_time_sorted['ISSUE DATETIME'])]

,NAME,ADDRESS,CITY,STATE,ZIP,LICENSE TYPE,LICENSE DESCRIPTION,ISSUE DATE,PHONE NUMBER,RESIDENCY,GENDER,AGE,ISSUE DATETIME
id,,,,,,,,,,,,,


In [205]:
license_df_time_sorted

,NAME,ADDRESS,CITY,STATE,ZIP,LICENSE TYPE,LICENSE DESCRIPTION,ISSUE DATE,PHONE NUMBER,RESIDENCY,GENDER,AGE,ISSUE DATETIME
id,,,,,,,,,,,,,
275220,"ROGERS, VERLA M",1646 KING WAY,WOODBURN,OR,97071,10160,ANNUAL SHELLFISH,12/1/2015,5039824022.0,Unknown,F,82.0,2015-12-01
249706,"MITCHELL, CHARLES L",1842 CHERRY ST,UMATILLA,OR,97882,10160,ANNUAL SHELLFISH,12/1/2015,5419226294.0,Unknown,M,71.0,2015-12-01
563279,"SCOTT, MICHAEL W",64643 BOONES BOROUGH DR,BEND,OR,97701.0,10574,PIONEER COMBINATION WITH COLUMBIA BASIN ENDORS...,12/1/2015,5413890112.0,Unknown,M,69.0,2015-12-01
250978,"MOORE, LAWRENCE H",5247 TERESA WAY,CENTRAL POINT,OR,97502,10565,DISABLED VET SHELLFISH,12/1/2015,5416644441.0,Unknown,M,60.0,2015-12-01
879870,"SIMMONS, WILLIAM R",623 DRIFTWOOD,EUGENE,OR,97402.0,10573,WHEELCHAIR ANGLING WITH COLUMBIA BASIN ENDORS...,12/1/2015,5414630518.0,Unknown,M,53.0,2015-12-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4384474,"KIENE, ROBERT N",1340 SW Gilson St,MCMINNVILLE,Oregon,97128,0101,Annual Combination License,01/01/2021,0,Resident,Male,79.0,2021-01-01
4248860,"Pearson, Dawn K",33040 E Lake Holm Dr SE,AUBURN,Washington,98092,0140A,Annual Shellfish License,1/1/2021,0.0,Non-Resident,Female,55.0,2021-01-01
5202549,NaN,NaN,NaN,NaN,NaN,0136B,Columbia River Basin Endorsement (Free),1/1/2021,NaN,Resident,Male,12.0,2021-01-01


Okay, last check. Let's call for unique values in some of these rows to make sure they are good.

In [218]:
set(license_df_time_sorted['LICENSE TYPE'].values);
# looks good

In [220]:
set(license_df_time_sorted['LICENSE DESCRIPTION'].values);
# looks good

In [221]:
set(license_df_time_sorted['RESIDENCY'])
# looks good

{NaT, 'Non-Resident', 'Resident', 'Uniform Services', 'Unknown', nan}

In [222]:
set(license_df_time_sorted['GENDER'])
# looks good. ay, they keep track of enbies, neat.

{'F', 'Female', 'M', 'Male', 'Non-Binary', 'Unknown', nan}

I think we are good! Let's save this, ugh.

In [223]:
license_df_time_sorted.to_csv(f'{dir}/OR_Hunting_Fishing_Licenses_2016_2020.csv')

Tomorrow, we plot.